# RFM Customer Segmentation 

    Segmentation : เพื่อแบ่งกลุ่มกลุ่มลูกค้าตามพฤติกรรมการซื้อของลูกค้าในแต่ละ transaction โดยที่  Model นี้อาจเหมาะสมกับสินค้าที่มีการซื้อซ้ำ
    เช่น อาหาร หรือเครื่องสำอางค์ เสื้อผ้า โดยมีตัวแปรสำคัญที่ใช้ในพิจารณา 3 ตัว คือ 
            1. Recency(R) คือ จำนวนวันที่ลูกค้าซื้อสินค้าล่าสุด นับจากวันที่เราพิจารณา
            2. Frequency(F) คือ ความถี่ที่ลูกค้ามาซื้อสินค้า มาซื้อบ่อยแค่ไหน
            3. Monetary(M) คือ จำนวนเงินลูกค้าซื้อตั้งแต่ transaction แรกจนถึงวันที่เราพิจารณา  ** โปรเจคนี้จะพิจารณาจากกำไร**
    

# 1. นำเข้า Library 

In [161]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

# 2. นำเข้า dataset 

In [162]:
#อ่านไฟล์ csv 
df = pd.read_csv("sales_data.csv")
df

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000101,In-Store,WARE-UHY1004,12/31/2017,5/31/2018,6/14/2018,6/19/2018,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,12/31/2017,5/31/2018,6/22/2018,7/2/2018,USD,14,20,196,27,3,0.075,3939.6,3348.660
2,SO - 000103,Distributor,WARE-UHY1004,12/31/2017,5/31/2018,6/21/2018,7/1/2018,USD,21,16,213,16,1,0.050,1775.5,781.220
3,SO - 000104,Wholesale,WARE-NMK1003,12/31/2017,5/31/2018,6/2/2018,6/7/2018,USD,28,48,107,23,8,0.075,2324.9,1464.687
4,SO - 000105,Distributor,WARE-NMK1003,4/10/2018,5/31/2018,6/16/2018,6/26/2018,USD,22,49,111,26,8,0.100,1822.4,1476.144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7986,SO - 0008087,In-Store,WARE-MKL1006,9/26/2020,12/30/2020,1/7/2021,1/14/2021,USD,9,41,339,29,1,0.075,234.5,121.940
7987,SO - 0008088,Online,WARE-NMK1003,9/26/2020,12/30/2020,1/2/2021,1/4/2021,USD,14,29,202,3,6,0.050,3202.6,1921.560
7988,SO - 0008089,Online,WARE-UHY1004,9/26/2020,12/30/2020,1/23/2021,1/26/2021,USD,14,32,241,35,5,0.200,3825.7,2792.761
7989,SO - 0008090,Online,WARE-NMK1003,9/26/2020,12/30/2020,1/20/2021,1/25/2021,USD,20,42,112,36,8,0.100,1072.0,804.000


# 3. เลือกเฉพาะ columns ที่สำคัญ และเปลี่ยนชื่อ columns ให้อ่านเข้าใจง่าย

    ข้อมูลที่เลือกมาสร้าง RFM Model มีดังนี้ 
1. _CustomerID รหัสลูกค้า
2. OrderNumber รหัสการสั่งซื้อ
3. OrderDate วันที่สั่งซื้อ
4. Order Quantity จำนวนชิ้น
5. Discount Applied อัตราส่วนลด
6. Unit Price ราคาขายต่อชิ้น
7. Unit Cost ต้นทุนต่อชิ้น

In [163]:
# เรียกดูชื่อ clolumns 
df.columns

Index(['OrderNumber', 'Sales Channel', 'WarehouseCode', 'ProcuredDate',
       'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', '_SalesTeamID',
       '_CustomerID', '_StoreID', '_ProductID', 'Order Quantity',
       'Discount Applied', 'Unit Price', 'Unit Cost'],
      dtype='object')

In [164]:
#เลือกcolumns ที่จะใช้งาน สำหรับ RFM 
# '_CustomerID', 'OrderNumber', 'OrderDate', 'Order Quantity', 'Discount Applied', 'Unit Price', 'Unit Cost'

df01 = df[['_CustomerID', 'OrderNumber', 'OrderDate', 'Order Quantity', 'Discount Applied', 'Unit Price', 'Unit Cost']]
df01

,_CustomerID,OrderNumber,OrderDate,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,15,SO - 000101,5/31/2018,5,0.075,1963.1,1001.181
1,20,SO - 000102,5/31/2018,3,0.075,3939.6,3348.660
2,16,SO - 000103,5/31/2018,1,0.050,1775.5,781.220
3,48,SO - 000104,5/31/2018,8,0.075,2324.9,1464.687
4,49,SO - 000105,5/31/2018,8,0.100,1822.4,1476.144
...,...,...,...,...,...,...,...
7986,41,SO - 0008087,12/30/2020,1,0.075,234.5,121.940
7987,29,SO - 0008088,12/30/2020,6,0.050,3202.6,1921.560
7988,32,SO - 0008089,12/30/2020,5,0.200,3825.7,2792.761
7989,42,SO - 0008090,12/30/2020,8,0.100,1072.0,804.000


In [165]:
#หาค่า Profit
#เก็บค่า Profit ลงไปใน dataframe df01  -->> df01["Profit"]
# Profit = รายรับ - ต้นทุน - ส่วนลด 
df01["Profit"] = (df01["Unit Price"] - df01["Unit Cost"] - df01["Unit Price"]*df01["Discount Applied"])*df01["Order Quantity"]
df01["Profit"]

0       4073.4325
1        886.4100
2        905.5050
3       5486.7640
4       1312.1280
          ...    
7986      94.9725
7987    6725.4600
7988    1338.9950
7989    1286.4000
7990    3371.7750
Name: Profit, Length: 7991, dtype: float64

In [166]:
df01

,_CustomerID,OrderNumber,OrderDate,Order Quantity,Discount Applied,Unit Price,Unit Cost,Profit
0,15,SO - 000101,5/31/2018,5,0.075,1963.1,1001.181,4073.4325
1,20,SO - 000102,5/31/2018,3,0.075,3939.6,3348.660,886.4100
2,16,SO - 000103,5/31/2018,1,0.050,1775.5,781.220,905.5050
3,48,SO - 000104,5/31/2018,8,0.075,2324.9,1464.687,5486.7640
4,49,SO - 000105,5/31/2018,8,0.100,1822.4,1476.144,1312.1280
...,...,...,...,...,...,...,...,...
7986,41,SO - 0008087,12/30/2020,1,0.075,234.5,121.940,94.9725
7987,29,SO - 0008088,12/30/2020,6,0.050,3202.6,1921.560,6725.4600
7988,32,SO - 0008089,12/30/2020,5,0.200,3825.7,2792.761,1338.9950
7989,42,SO - 0008090,12/30/2020,8,0.100,1072.0,804.000,1286.4000


In [167]:
# เลือก columns ที่จำเป็น 
# "_CustomerID", "OrderNumber", "OrderDate", "Profit"
df01 = df01[["_CustomerID", "OrderNumber", "OrderDate", "Profit"]]
df01

,_CustomerID,OrderNumber,OrderDate,Profit
0,15,SO - 000101,5/31/2018,4073.4325
1,20,SO - 000102,5/31/2018,886.4100
2,16,SO - 000103,5/31/2018,905.5050
3,48,SO - 000104,5/31/2018,5486.7640
4,49,SO - 000105,5/31/2018,1312.1280
...,...,...,...,...
7986,41,SO - 0008087,12/30/2020,94.9725
7987,29,SO - 0008088,12/30/2020,6725.4600
7988,32,SO - 0008089,12/30/2020,1338.9950
7989,42,SO - 0008090,12/30/2020,1286.4000


In [168]:
 # ตั้งชื่อใช้ เป็น df 
# เปลี่ยนชื่อแต่ละ column ให้อ่านง่าย

df = df01.rename(columns= {
            "_CustomerID" : "Customer_ID",
            "OrderNumber" : "Order_Number",
            "OrderDate" : "Order_Date",
            "Profit" : "Profit"
            })

df

,Customer_ID,Order_Number,Order_Date,Profit
0,15,SO - 000101,5/31/2018,4073.4325
1,20,SO - 000102,5/31/2018,886.4100
2,16,SO - 000103,5/31/2018,905.5050
3,48,SO - 000104,5/31/2018,5486.7640
4,49,SO - 000105,5/31/2018,1312.1280
...,...,...,...,...
7986,41,SO - 0008087,12/30/2020,94.9725
7987,29,SO - 0008088,12/30/2020,6725.4600
7988,32,SO - 0008089,12/30/2020,1338.9950
7989,42,SO - 0008090,12/30/2020,1286.4000


# 4. Data preparation

In [169]:
# Check missing values

df.isnull().sum() #ไม่มีค่า null

Customer_ID     0
Order_Number    0
Order_Date      0
Profit          0
dtype: int64

In [170]:
#Check data type
df.dtypes

Customer_ID       int64
Order_Number     object
Order_Date       object
Profit          float64
dtype: object

In [171]:
#เปลี่ยน Order_Date เป็น datetime
df["Order_Date"] = pd.to_datetime(df["Order_Date"])
df["Order_Date"]

0      2018-05-31
1      2018-05-31
2      2018-05-31
3      2018-05-31
4      2018-05-31
          ...    
7986   2020-12-30
7987   2020-12-30
7988   2020-12-30
7989   2020-12-30
7990   2020-12-30
Name: Order_Date, Length: 7991, dtype: datetime64[ns]

In [172]:
# check ค่า revenue ติดลบมั้ย ขาดทุน
df[df["Profit"] < 0]

,Customer_ID,Order_Number,Order_Date,Profit
20,46,SO - 000121,2018-06-01,-1549.040
58,47,SO - 000159,2018-06-06,-171.520
59,33,SO - 000160,2018-06-07,-778.540
88,5,SO - 000189,2018-06-11,-1976.500
136,50,SO - 000237,2018-06-16,-258.285
...,...,...,...,...
7909,36,SO - 0008010,2020-12-21,-704.304
7915,7,SO - 0008016,2020-12-22,-48.240
7936,47,SO - 0008037,2020-12-24,-548.864
7938,33,SO - 0008039,2020-12-24,-167.500


In [173]:
df["Order_Date"].max()

Timestamp('2020-12-30 00:00:00')

# 5.คำนวณค่า RFM Metrics และสร้างกรอบวันที่ต้องการวิเคราะห์

In [174]:
# เนื่องจาก df["Order_Date"].max()  คือ 2020-12-30
# ดังนั้นจึงกำหนด วันที่พิจารณา analyze_date คือ 2021-01-01
analyze_date = pd.to_datetime('2021-01-01')

In [175]:
df.columns

Index(['Customer_ID', 'Order_Number', 'Order_Date', 'Profit'], dtype='object')

In [176]:
# Grouping ลูกค้าที่มีชื่อเดียวกัน (customer_name)
rfm_dataset = df.groupby(["Customer_ID"]).agg({
        "Order_Date" : lambda x :  (analyze_date - x.max()).days ,
        "Order_Number" : lambda x : x.nunique(), #นับ order ที่ไม่ซ้ำกัน
        "Profit" : lambda x : x.sum() 
    })

In [177]:
rfm_dataset

,Order_Date,Order_Number,Profit
Customer_ID,,,
1,9,152,335933.6115
2,9,135,327409.1345
3,10,181,466220.1365
4,5,167,526981.0630
5,30,159,445632.7450
6,4,143,392141.0840
7,3,153,414155.9775
8,5,142,338000.0590
9,8,171,500166.4570


In [178]:
# ตรวจ data type ของ columns Order_Date ต้องเป็น int
rfm_dataset.dtypes

Order_Date        int64
Order_Number      int64
Profit          float64
dtype: object

In [179]:
# เปลี่ยนชื่อ columns 
rfm_dataset.rename(
    columns= {
        "Order_Date" : "Recency",
        "Order_Number" : "Frequency",
        "Profit" : "Monetary"
    }, inplace = True)

In [180]:
rfm_dataset.head(3)

,Recency,Frequency,Monetary
Customer_ID,,,
1,9,152,335933.6115
2,9,135,327409.1345
3,10,181,466220.1365


# 6. สร้าง R F M columns คำนวณ RFM_Score และ RF_Score ตามลำดับ

In [181]:
#rfm_dataset = rfm_dataset.assign(R = R.values, F = F.values, M = M.values)

In [182]:
# สร้าง R F M columns 
rfm_dataset["R"] = pd.qcut(rfm_dataset["Recency"], q = 5, labels= range(5, 0, -1)) # [5,4,3,2,1]
rfm_dataset["F"] = pd.qcut(rfm_dataset["Frequency"], q = 5, labels= range(1, 6)) #[1,2,3,4,5]
rfm_dataset["M"] = pd.qcut(rfm_dataset["Frequency"], q = 5, labels= range(1, 6)) #[1,2,3,4,5]

In [183]:
rfm_dataset

,Recency,Frequency,Monetary,R,F,M
Customer_ID,,,,,,
1,9,152,335933.6115,2,2,2
2,9,135,327409.1345,2,1,1
3,10,181,466220.1365,2,5,5
4,5,167,526981.0630,3,4,4
5,30,159,445632.7450,1,3,3
6,4,143,392141.0840,4,1,1
7,3,153,414155.9775,5,2,2
8,5,142,338000.0590,3,1,1
9,8,171,500166.4570,2,4,4


In [184]:
# คำนวณ RFM_Score
# concept คือ การเอา R, F, M มาต่อกันเป็น string

# rfm_dataset["RFM_Group"] = rfm_dataset["R"].astype(str) + rfm_dataset["F"].astype(str) + rfm_dataset["M"].astype(str)

rfm_dataset["RFM_Score"] = rfm_dataset[["R", "F", "M"]].apply(lambda y : "".join(y.astype(str)), axis = 1)

In [185]:
rfm_dataset

,Recency,Frequency,Monetary,R,F,M,RFM_Score
Customer_ID,,,,,,,
1,9,152,335933.6115,2,2,2,222
2,9,135,327409.1345,2,1,1,211
3,10,181,466220.1365,2,5,5,255
4,5,167,526981.0630,3,4,4,344
5,30,159,445632.7450,1,3,3,133
6,4,143,392141.0840,4,1,1,411
7,3,153,414155.9775,5,2,2,522
8,5,142,338000.0590,3,1,1,311
9,8,171,500166.4570,2,4,4,244


# RFMScore

In [186]:
# หา RFM Score 
# concept : เอา R F M มาบวกกัน

# rfm_dataset["RFM_Score"] = rfm_dataset[["R", "F", "M"]].sum(axis = 1)

-  ใน Pandas sum(axis=1) หมายถึงการหาผลรวมของแต่ละแถว (row) ใน DataFrame

   โดยที่:

-  axis=0 (หรือไม่ระบุ axis) จะหมายถึงการหาผลรวมตามแนวแกน columns (แนวตั้ง) ดังนั้น df.sum(axis=0) จะคำนวณผลรวมของแต่ละคอลัมน์
 - axis=1 จะหมายถึงการหาผลรวมตามแนวแกน rows (แนวนอน) ดังนั้น df.sum(axis=1) จะคำนวณผลรวมของแต่ละแถว

In [187]:
rfm_dataset

,Recency,Frequency,Monetary,R,F,M,RFM_Score
Customer_ID,,,,,,,
1,9,152,335933.6115,2,2,2,222
2,9,135,327409.1345,2,1,1,211
3,10,181,466220.1365,2,5,5,255
4,5,167,526981.0630,3,4,4,344
5,30,159,445632.7450,1,3,3,133
6,4,143,392141.0840,4,1,1,411
7,3,153,414155.9775,5,2,2,522
8,5,142,338000.0590,3,1,1,311
9,8,171,500166.4570,2,4,4,244


# 7. Mapping ข้อมูล 

    Segment Description  
  
- Champions Bought recently, buy often and spend the most
- Loyal Customers Buy on a regular basis. Responsive to promotions.
- Potential Loyalist Recent customers with average frequency.
- Recent Customers Bought most recently, but not often.
- Promising Recent shoppers, but haven’t spent much.
- Customers Needing Attention Above average recency, frequency and monetary values. May not have bought very recently though.
- About To Sleep Below average recency and frequency. Will lose them if not reactivated.
- At Risk Purchased often but a long time ago. Need to bring them back!
- Can’t Lose Them Used to purchase frequently but haven’t returned for a long time.
- Hibernating Last purchase was long back and low number of orders. May be lost.
- Lost Lowest recency, frequency, and monetary score


In [188]:
# สร้าง column RF_Score
rfm_dataset["RF_Score"] = rfm_dataset[["R", "F"]].apply(lambda y : "".join(y.astype(str)), axis = 1)

In [189]:
rfm_dataset

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score
Customer_ID,,,,,,,,
1,9,152,335933.6115,2,2,2,222,22
2,9,135,327409.1345,2,1,1,211,21
3,10,181,466220.1365,2,5,5,255,25
4,5,167,526981.0630,3,4,4,344,34
5,30,159,445632.7450,1,3,3,133,13
6,4,143,392141.0840,4,1,1,411,41
7,3,153,414155.9775,5,2,2,522,52
8,5,142,338000.0590,3,1,1,311,31
9,8,171,500166.4570,2,4,4,244,24


In [190]:
segments_map = {
        r'5[4-5]': 'Champions',
        r'[3-4][4-5]': 'Loyal Customers',
        r'[4-5][2-3]': 'Potential Loyalist',
        r'51': 'New Customers',
        r'41': 'Promising',
        r'33': 'Need Attention',
        r'3[1-2]': 'About To Sleep',
        r'[1-2][3-4]': 'At Risk',
        r'[1-2]5': 'Can’t Lose Them',
        r'[1-2][1-2]': 'Hibernating'
          }

In [192]:
rfm_dataset

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score
Customer_ID,,,,,,,,
1,9,152,335933.6115,2,2,2,222,22
2,9,135,327409.1345,2,1,1,211,21
3,10,181,466220.1365,2,5,5,255,25
4,5,167,526981.0630,3,4,4,344,34
5,30,159,445632.7450,1,3,3,133,13
6,4,143,392141.0840,4,1,1,411,41
7,3,153,414155.9775,5,2,2,522,52
8,5,142,338000.0590,3,1,1,311,31
9,8,171,500166.4570,2,4,4,244,24


In [194]:
#แทนค่า
rfm_dataset["Segment"] = rfm_dataset["RF_Score"].replace(segments_map, regex = True)

In [195]:
rfm_dataset

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
1,9,152,335933.6115,2,2,2,222,22,Hibernating
2,9,135,327409.1345,2,1,1,211,21,Hibernating
3,10,181,466220.1365,2,5,5,255,25,Can’t Lose Them
4,5,167,526981.0630,3,4,4,344,34,Loyal Customers
5,30,159,445632.7450,1,3,3,133,13,At Risk
6,4,143,392141.0840,4,1,1,411,41,Promising
7,3,153,414155.9775,5,2,2,522,52,Potential Loyalist
8,5,142,338000.0590,3,1,1,311,31,About To Sleep
9,8,171,500166.4570,2,4,4,244,24,At Risk


In [196]:
#save ออกไป
rfm_dataset.to_csv("1.RFM_ Customer_Segmentation .csv")

# 1. ลูกค้าแต่ละ segmentation มีเท่าไหร่

In [197]:
rfm_dataset.shape

(50, 9)

In [198]:
rfm_dataset["Segment"].value_counts()

Hibernating           10
Potential Loyalist     9
Loyal Customers        7
At Risk                7
Champions              5
Can’t Lose Them        3
Promising              3
About To Sleep         3
New Customers          2
Need Attention         1
Name: Segment, dtype: int64

# 2. Check segments mean, min and max

In [199]:
rfm_dataset.groupby(["Segment"])[['Recency', 'Frequency', 'Monetary']].agg(['mean', 'min', 'max']).sort_values(by=[('Recency', 'mean')], ascending=True)
                                                                                                            

Recency           Frequency                 Monetary  \
                         mean min max        mean  min  max           mean   
Segment                                                                      
Champions            2.600000   2   3  178.200000  164  210  501262.188400   
New Customers        3.000000   3   3  139.500000  135  144  369949.277000   
Potential Loyalist   3.333333   2   4  156.777778  152  161  416588.010500   
Promising            4.000000   4   4  141.666667  140  143  411843.740500   
About To Sleep       5.000000   5   5  149.666667  142  156  372538.804667   
Need Attention       5.000000   5   5  157.000000  157  157  381450.028000   
Loyal Customers      5.142857   4   6  173.571429  162  186  482067.694357   
Hibernating         10.500000   7  23  148.500000  135  153  362206.200900   
Can’t Lose Them     11.666667  10  15  176.333333  172  181  462536.521167   
At Risk             14.000000   8  30  164.285714  158  171  458726.066857   

                                              
                            min          max  
Segment                                       
Champions           435206.0710  616719.2550  
New Customers       336959.7835  402938.7705  
Potential Loyalist  348209.0500  495444.2635  
Promising           392141.0840  441668.3550  
About To Sleep      338000.0590  407145.0305  
Need Attention      381450.0280  381450.0280  
Loyal Customers     441003.2795  534027.3860  
Hibernating         327409.1345  425241.0940  
Can’t Lose Them     424970.7490  496418.6780  
At Risk             435122.1870  500166.4570

# 3. Who are the best customers (Champions)?

In [200]:
rfm_dataset[ rfm_dataset["Segment"] == "Champions"]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
12,3,210,616719.2550,5,5,5,555,55,Champions
19,3,165,443231.8335,5,4,4,544,54,Champions
21,3,164,479383.0905,5,4,4,544,54,Champions
29,2,179,531770.6920,5,5,5,555,55,Champions
32,2,173,435206.0710,5,5,5,555,55,Champions


    Champions: 
- สุดยอดลูกค้า ดีสุดในทุกด้าน 
- ถือเป็นกลุ่มลูกค้าที่เราควรรักษาไว้ให้ได้ 
- โดยการออก Campaigns พิเศษ และการทำการตลาดแบบเฉพาะกลุ่มบุคคล (Personalize)

# 4. Loyal Customers 

In [201]:
rfm_dataset[ rfm_dataset["Segment"] == "Loyal Customers"]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
4,5,167,526981.0630,3,4,4,344,34,Loyal Customers
11,6,178,487614.2415,3,5,5,355,35,Loyal Customers
13,4,171,441003.2795,4,4,4,444,44,Loyal Customers
17,6,175,534027.3860,3,5,5,355,35,Loyal Customers
18,6,186,451637.7540,3,5,5,355,35,Loyal Customers
25,5,162,461601.9940,3,4,4,344,34,Loyal Customers
39,4,176,471608.1425,4,5,5,455,45,Loyal Customers


    Loyal Customers: 
- เคยเป็นลูกค้าที่ซื้อบ่อย ซื้อเยอะ แต่ไม่ซื้อมานานแล้ว 
- ทำให้เราควรเน้น Promotions ที่กระตุ้นการซื้อให้เพิ่มขึ้น หรือให้สินค้าทดลองเพื่อให้กลับมาซื้อสินค้าอีกครั้ง

# 5. Potential Loyalist

In [202]:
rfm_dataset[ rfm_dataset["Segment"] == "Potential Loyalist"]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
7,3,153,414155.9775,5,2,2,522,52,Potential Loyalist
30,4,159,442372.3910,4,3,3,433,43,Potential Loyalist
31,4,152,398616.8350,4,2,2,422,42,Potential Loyalist
33,4,156,495444.2635,4,2,2,422,42,Potential Loyalist
36,4,156,441225.9540,4,2,2,422,42,Potential Loyalist
41,2,161,403693.2910,5,3,3,533,53,Potential Loyalist
42,2,161,427159.6055,5,3,3,533,53,Potential Loyalist
44,3,156,348209.0500,5,2,2,522,52,Potential Loyalist
46,4,157,378414.7270,4,3,3,433,43,Potential Loyalist


    Potential Loyalist: 
- เพิ่งมาเป็นลุกค้า ซื้อมากกว่า 1 ครั้ง  ซื้อเยอะ
- กลุ่มนี้หลังจากที่มีการซื้อสินค้า ให้รีบทำการติดต่อกลับทันที เพื่อให้ความสำคัญ 
- และเก็บข้อมูลเพื่อเสนอขายสินค้าที่ตรงกับความต้องการเพื่อกระตุ้นการซื้อได้

# 6. New Customers:

In [203]:
rfm_dataset[( rfm_dataset["Segment"] == "New Customers")]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
16,3,135,402938.7705,5,1,1,511,51,New Customers
27,3,144,336959.7835,5,1,1,511,51,New Customers


    New Customers: 
- ลูกค้าใหม่ เพิ่งมาซื้อ จ่ายน้อย 
- ให้ติดต่อกลับไปเพื่อให้บริการหลังการขาย หรือส่งข้อมูลที่เป็นประโยชน์ต่อการใช้สินค้าได้อย่างเหมาะสมมากยิ่งขึ้น

# 7. Promising

In [204]:
rfm_dataset[( rfm_dataset["Segment"] == "Promising")]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
6,4,143,392141.0840,4,1,1,411,41,Promising
15,4,142,441668.3550,4,1,1,411,41,Promising
22,4,140,401721.7825,4,1,1,411,41,Promising


    Promising: 
 - ลูกค้านานๆมาที แต่กลับมาอยู่นะ ซื้อจางๆ 
 - ต้องกระตุ้นด้วยการทำระบบสมาชิกสะสมแต้ม เพื่อดึงมาเป็น Loyal Customers

# 8. Need Attention

In [205]:
rfm_dataset[( rfm_dataset["Segment"] == "Need Attention")]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
14,5,157,381450.028,3,3,3,333,33,Need Attention


    Need Attention: 
- เคยมาซื้อ แล้วหายไป ตอนนี้กลับมาซื้ออีก
- มาแล้วมาถี่ 
- จ่ายเยอะ 
- ซึ่งเราสามารถส่งข้อมูลผ่านช่องทางติดต่อที่ลูกค้ากลุ่มนี้ให้ไว้เพื่อดึงให้กลับมาซื้อ 
- โดยส่งข้อมูลของสินค้าที่ลูกค้าเคยซื้อ หรือสินค้าใกล้เคียง 
- เพื่อแสดงความใส่ใจ หรือเสนอส่วนลดแบบมีกรอบเวลา เพื่อให้กลับมาในช่วงเวลาที่ตั้งกรอบไว้

# 9. About to Sleep

In [206]:
rfm_dataset[( rfm_dataset["Segment"] == "About To Sleep")]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
8,5,142,338000.0590,3,1,1,311,31,About To Sleep
43,5,151,372471.3245,3,2,2,322,32,About To Sleep
45,5,156,407145.0305,3,2,2,322,32,About To Sleep


    About to Sleep: 
- หายไปนานแล้ว 
- เคยซื้อบ่อยด้วย 
- ซื้อน้อย
- ให้ติดต่อกลับไปบ้างเพื่อเสนอโปรโมชั่น และสินค้าใหม่

# 10. At Risk 

In [207]:
rfm_dataset[( rfm_dataset["Segment"] == "At Risk")]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
5,30,159,445632.7450,1,3,3,133,13,At Risk
9,8,171,500166.4570,2,4,4,244,24,At Risk
10,15,158,435122.1870,1,3,3,133,13,At Risk
20,9,167,439147.9490,2,4,4,244,24,At Risk
23,12,164,449782.2895,1,4,4,144,14,At Risk
47,8,168,442845.8465,2,4,4,244,24,At Risk
50,16,163,498384.9940,1,4,4,144,14,At Risk


    At Risk: 
- หายไปนานมากกกแล้ว 
- เคยซื้อเยอะ
- รายนี้ต้อง Contact ด่วนๆ เลย ไม่งั้นหายไปเลยแน่ 
- โดยอาจจะส่งข้อมูลสินค้าที่เป็นที่นิยมในปัจจุบัน พร้อมโปรโมชั่น เผื่อเขาจะกลับมา

# 11. Can’t Lose Them

In [208]:
rfm_dataset[( rfm_dataset["Segment"] == "Can’t Lose Them")]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
3,10,181,466220.1365,2,5,5,255,25,Can’t Lose Them
34,15,176,496418.6780,1,5,5,155,15,Can’t Lose Them
48,10,172,424970.7490,2,5,5,255,25,Can’t Lose Them


    Can’t Lose Them: 
- ไม่ซื้อนานแล้ว 
- เคยซื้อบ่อยๆ 
- ซื้อเยอะ 
     
- ควรติดต่อไปโดยตรงเพื่อให้ความสำคัญ พร้อมกับส่ง Promotions พิเศษไปให้

# 12. Hibernating

In [209]:
rfm_dataset[( rfm_dataset["Segment"] == "Hibernating")]

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
1,9,152,335933.6115,2,2,2,222,22,Hibernating
2,9,135,327409.1345,2,1,1,211,21,Hibernating
24,23,151,352505.5255,1,2,2,122,12,Hibernating
26,11,153,375766.6860,1,2,2,122,12,Hibernating
28,8,145,348495.4750,2,1,1,211,21,Hibernating
35,10,145,345844.5865,2,1,1,211,21,Hibernating
37,11,152,425241.0940,1,2,2,122,12,Hibernating
38,8,150,350687.6480,2,1,1,211,21,Hibernating
40,9,150,406016.9180,2,1,1,211,21,Hibernating


    Hibernating:
- ซื้อน้อย ไม่บ่อย ไม่ซื้อนานแล้ว
- อาจจะต้องวัดใจ ให้ Promotion แรงๆ เพื่อกระตุ้นให้เขากลับมาซื้อ แต่ถ้าเขาไม่กลับมาก็คงต้องทำใจ และสำรวจข้อบกพร่องของผลิตภัณฑ์ หรือคู่แข่ง

In [210]:
rfm_dataset

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RF_Score,Segment
Customer_ID,,,,,,,,,
1,9,152,335933.6115,2,2,2,222,22,Hibernating
2,9,135,327409.1345,2,1,1,211,21,Hibernating
3,10,181,466220.1365,2,5,5,255,25,Can’t Lose Them
4,5,167,526981.0630,3,4,4,344,34,Loyal Customers
5,30,159,445632.7450,1,3,3,133,13,At Risk
6,4,143,392141.0840,4,1,1,411,41,Promising
7,3,153,414155.9775,5,2,2,522,52,Potential Loyalist
8,5,142,338000.0590,3,1,1,311,31,About To Sleep
9,8,171,500166.4570,2,4,4,244,24,At Risk


In [211]:
segments_counts = rfm_dataset["Segment"].value_counts().sort_values( ascending = True)
segments_counts 

Need Attention         1
New Customers          2
Can’t Lose Them        3
Promising              3
About To Sleep         3
Champions              5
Loyal Customers        7
At Risk                7
Potential Loyalist     9
Hibernating           10
Name: Segment, dtype: int64

In [212]:
segments_counts.to_csv("segments_counts.csv")